# 4. Validation of E2F3 amplification 

In [ ]:
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
from plotting_settings import PLOTTING_PARAMS
from scipy.stats import chi2_contingency, mannwhitneyu

warnings.filterwarnings("ignore")
sys.path.insert(1, "../../helper_functions")

In [ ]:
def extract_spots(adata, x1, x2, y1, y2):
    extracted_spots = []
    for i, spot in enumerate(adata.obsm["spatial"].tolist()):
        x, y = spot
        if (x > x1) and (x < x2) and (y > y1) and (y < y2):
            extracted_spots.append(i)

    return extracted_spots

## E2F3 results from FISH

In [ ]:
b42_crops = [
    ("right", [13000, 23000, 4000, 20000]),
    ("left", [3000, 13000, 5000, 22000]),
]
b123_crops = [("top", [3000, 24000, 0, 12000]), ("bottom", [3000, 24000, 12000, 27000])]

In [ ]:
b123 = sc.read_h5ad(f"../../data/B123.h5ad")
b42 = sc.read_h5ad(f"../../data/B42.h5ad")

In [ ]:
b42_right = extract_spots(
    b42,
    x1=b42_crops[0][1][0],
    x2=b42_crops[0][1][1],
    y1=b42_crops[0][1][2],
    y2=b42_crops[0][1][3],
)
b42_left = extract_spots(
    b42,
    x1=b42_crops[1][1][0],
    x2=b42_crops[1][1][1],
    y1=b42_crops[1][1][2],
    y2=b42_crops[1][1][3],
)

b42_right = b42[b42_right, :].copy()
b42_left = b42[b42_left, :].copy()

b123_top = extract_spots(
    b123,
    x1=b123_crops[0][1][0],
    x2=b123_crops[0][1][1],
    y1=b123_crops[0][1][2],
    y2=b123_crops[0][1][3],
)
b123_bottom = extract_spots(
    b123,
    x1=b123_crops[1][1][0],
    x2=b123_crops[1][1][1],
    y1=b123_crops[1][1][2],
    y2=b123_crops[1][1][3],
)

b123_top = b123[b123_top, :].copy()
b123_bottom = b123[b123_bottom, :].copy()

#### B42

In [ ]:
sns.set_theme(style="white", rc=PLOTTING_PARAMS)
fig, axes = plt.subplots(2, 2, figsize=(6, 6))
sc.pl.spatial(
    b42_left,
    color=None,
    size=1.5,
    title="B42 left",
    show=False,
    ax=axes[0, 0],
    frameon=False,
)
sc.pl.spatial(
    b42_left,
    color="E2F3",
    size=1.5,
    vmax=1.4,
    show=False,
    ax=axes[0, 1],
    frameon=False,
    cmap="viridis",
)
sc.pl.spatial(
    b42_right,
    color=None,
    size=1.5,
    title="B42 right",
    show=False,
    ax=axes[1, 0],
    frameon=False,
)
sc.pl.spatial(
    b42_right,
    color="E2F3",
    size=1.5,
    vmax=1.4,
    show=False,
    ax=axes[1, 1],
    frameon=False,
    cmap="viridis",
)

plt.tight_layout()

plt.savefig("suppfig7A_spatial_FISH_E2F3_B42.svg", dpi=300)

In [ ]:
b42_left_df = sc.get.obs_df(b42_left, keys=["E2F3"])
b42_left_df["Area"] = "Left"
b42_right_df = sc.get.obs_df(b42_right, keys=["E2F3"])
b42_right_df["Area"] = "Right"

b42_joint = pd.concat([b42_left_df, b42_right_df])

plt.figure(figsize=(6, 4))
sns.boxplot(b42_joint, x="Area", y="E2F3")
sns.despine(top=True, right=True, left=False, bottom=False)

plt.savefig("suppfig_7C_boxplot_E2F3_exp_B42.svg", dpi=300, bbox_inches="tight")

In [ ]:
mannwhitneyu(
    b42_joint[b42_joint["Area"] == "Left"]["E2F3"],
    b42_joint[b42_joint["Area"] == "Right"]["E2F3"],
)

#### B123

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(6, 6))
sc.pl.spatial(
    b123_top,
    color=None,
    size=1.5,
    title="B123 top",
    show=False,
    ax=axes[0, 0],
    frameon=False,
)
sc.pl.spatial(
    b123_top,
    color="E2F3",
    size=1.5,
    vmax=1.4,
    show=False,
    ax=axes[0, 1],
    frameon=False,
    cmap="viridis",
)
sc.pl.spatial(
    b123_bottom,
    color=None,
    size=1.5,
    title="B123 bottom",
    show=False,
    ax=axes[1, 0],
    frameon=False,
)
sc.pl.spatial(
    b123_bottom,
    color="E2F3",
    size=1.5,
    vmax=1.4,
    show=False,
    ax=axes[1, 1],
    frameon=False,
    cmap="viridis",
)

plt.tight_layout()

plt.savefig("suppfig_7D_spatial_FISH_E2F3_B123.svg", dpi=300)

In [ ]:
b123_top_df = sc.get.obs_df(b123_top, keys=["E2F3"])
b123_top_df["Area"] = "Top"
b123_bottom_df = sc.get.obs_df(b123_bottom, keys=["E2F3"])
b123_bottom_df["Area"] = "Bottom"

b123_joint = pd.concat([b123_top_df, b123_bottom_df])

plt.figure(figsize=(6, 4))
sns.boxplot(b123_joint, x="Area", y="E2F3")
sns.despine(top=True, right=True, left=False, bottom=False)

plt.savefig("suppfig_7F_boxplot_E2F3_exp_B123.svg", dpi=300, bbox_inches="tight")

In [ ]:
mannwhitneyu(
    b123_joint[b123_joint["Area"] == "Top"]["E2F3"],
    b123_joint[b123_joint["Area"] == "Bottom"]["E2F3"],
)

## FISH

#### Just HLA + 8 signals

In [ ]:
b123_table = np.array([[50, 51], [41, 60]])
b42_table = np.array([[66, 44], [67, 33]])

In [ ]:
chi2_contingency(b123_table)

In [ ]:
chi2_contingency(b42_table)

In [ ]:
weight_counts = {
    "E2F3 high-level\namplification": np.array([50 / 101, 41 / 101]),
    "E2F3 non-high-level\namplification": np.array([51 / 101, 60 / 101]),
}
df = pd.DataFrame.from_dict(weight_counts, orient="index", columns=["Top", "Bottom"]).T
# df = df[["non-HLA", "HLA"]]
ax = df.plot.bar(
    stacked=True, width=0.9, figsize=(6, 4), color=sns.color_palette("colorblind")[5:]
)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.title("B123")
plt.xticks(rotation=0)
plt.xlabel("Area")
plt.ylabel("Fraction of cells")
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))

plt.savefig("suppfig_7E_barplot_FISH_E2F3_B123.svg", dpi=300, bbox_inches="tight")

In [ ]:
weight_counts = {
    "E2F3 high-level\namplification": np.array([66 / 110, 67 / 100]),
    "E2F3 non-high-level\namplification": np.array([44 / 110, 33 / 100]),
}
df = pd.DataFrame.from_dict(weight_counts, orient="index", columns=["Left", "Right"]).T
# df = df[["non-HLA", "HLA"]]
ax = df.plot.bar(
    stacked=True, width=0.9, figsize=(6, 4), color=sns.color_palette("colorblind")[5:]
)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.title("B42")
plt.xlabel("Area")
plt.xticks(rotation=0)
plt.ylabel("Fraction of cells")
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))

plt.savefig("suppfig_7B_barplot_FISH_E2F3_B42.svg", dpi=300, bbox_inches="tight")